<a href="https://colab.research.google.com/github/geoaigroup/Aerial-SAM/blob/main/AerialSAM_GEOAI_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Samples used in this demo is from the WHU Building Dataset

WHU dataset : https://paperswithcode.com/dataset/whu-building-dataset

# Object masks from prompts with SAM
This section was prepared by Ali Mayladan.

The Segment Anything Model (SAM) predicts object masks given prompts that indicate the desired object. The model first converts the image into an image embedding that allows high quality masks to be efficiently produced from a prompt.

The `SamPredictor` class provides an easy interface to the model for prompting the model. It allows the user to first set an image using the `set_image` method, which calculates the necessary image embeddings. Then, prompts can be provided via the `predict` method to efficiently predict masks from those prompts. The model can take as input both point and box prompts, as well as masks from the previous iteration of prediction.

## Environment Set-up

If running locally using jupyter, first install `segment_anything` in your environment using the [installation instructions](https://github.com/facebookresearch/segment-anything#installation) in the repository. If running from Google Colab, set `using_colab=True` below and run the cell. In Colab, be sure to select 'GPU' under 'Edit'->'Notebook Settings'->'Hardware accelerator'.

In [ ]:
!wget https://github.com/geoaigroup/Aerial-SAM/raw/main/resources/data.zip
!wget https://github.com/geoaigroup/Aerial-SAM/raw/main/resources/pred_shapefile.zip
!unzip data.zip
!unzip pred_shapefile

In [ ]:
using_colab = True

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
    !pip install geopandas
    !pip install rasterio
    !git clone https://github.com/geoaigroup/buildingsSAM.git


## Set-up

Necessary imports and helper functions for displaying points, boxes, and masks.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import geopandas as gpd
import os
import json
import glob
from tqdm import tqdm
import shapely.geometry as sg
from shapely import affinity
from shapely.geometry import Point, Polygon
import random
from PIL import Image, ImageDraw
import rasterio
from rasterio.features import geometry_mask
#from metrics import DiceScore,IoUScore
import pandas as pd
import gc
import shutil
import fiona
import json
from buildingsSAM import utils
from buildingsSAM.evaluate import cal_scores
from buildingsSAM.pred_SAM import SAM

In [ ]:
def main(CNN="",prompt_type="",sam=None):
    score_list = []
    # ff = gpd.read_file(pred)
    ids = [f for f in os.listdir(orig_shp)]
    for name in tqdm(ids):
        print(name)
        print("Checking")
        flag=0
        if glob.glob(output_dir + "/" + name + "/" + name + ".shp" ) or glob.glob(output_dir + "/" + name + "/" + name + ".png" ):
            print("Found")
            continue

        tile_boxes = []
        try:
            image = cv2.imread(images + "/" + name+'.png')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(e)
            print(name)

        if CNN=="multiclassUnet":

            if name in os.listdir(orig_shp):
                gt = gpd.read_file(orig_shp + "/" + name)
                if len(gt["geometry"]) == 0:
                    continue
            else:
                continue
            predic = ff.loc[ff["ImageId"] == name]

        elif CNN=="DCNN":
            predic = gpd.read_file(pred+"/"+dirr+"/"+name)

        geo = predic["geometry"]

        if len(geo) == 0:
            continue

        input_point=[]
        input_label=[]

        ###creating boxes
        if prompt_type=="box":
            tile_boxes=utils.create_boxes(geo)
            input_boxes=torch.tensor(tile_boxes).cuda()

        ##Skeleton
        if prompt_type=="skeleton":
            with open(skeleton_points, 'r') as json_file:
                data = json.load(json_file)
            matching_items = []
            for item in data:
                if item['id'] == name:
                    matching_items.append(item)

            input_point=torch.Tensor(matching_items[0]['input_points']).cuda()
            input_label=torch.Tensor(matching_items[0]['input_labels']).cuda().long()

        x = torch.from_numpy(image.transpose(2, 0, 1)).float().cuda()
        pred_mask=sam.predictSAM(x=x,image=image,input_point=input_point,input_label=input_label,input_boxes=input_boxes,flag=flag)
        os.makedirs(score_dir, exist_ok=True)
        os.makedirs(output_dir + "/" + f"{name}", exist_ok=True)
        scores=cal_scores(output_dir,score_dir)
        scores.micro_match_iou(pred_mask,name,gt,score_list,image,tile_boxes,geo=geo)
    scores.macro_score()


In [ ]:
# Paths
images = "data/images"
output_dir = "data/output"
pred = "pred_shapefile"
orig_shp="data/orig_shp"
score_dir = "data/scores"
skeleton_points="data/points.json"

In [ ]:

#loading SAM Model
sam=SAM()
#load Multiclass Unet CNN prediction file
ff = gpd.read_file(pred)

#Run SAM prediction with box prompt
main(CNN="multiclassUnet",prompt_type="box",sam=sam)

#Run SAM prediction with skeleton prompt
main(CNN="multiclassUnet",prompt_type="skeleton",sam=sam)

In [ ]:
#loading SAM Model
sam=SAM()
pred = "data/DCNN_pred_shapefile"
main(CNN="DCNN",prompt_type="box",sam=sam)

LangSAM : https://github.com/luca-medeiros/lang-segment-anything

GroundingDINO: https://github.com/IDEA-Research/GroundingDINO

Segment Anything : https://github.com/facebookresearch/segment-anything

**Language Segment-Anything (LangSAM)** is an open-source project that combines the power of instance segmentation and text prompts to generate masks for specific objects in images. Built on the recently released Meta model, segment-anything, and the GroundingDINO detection model, it's an easy-to-use and effective tool for object detection and image segmentation.

This section was prepared by Hasan Moughnieh.

In [ ]:
!wget https://github.com/geoaigroup/geoaigroup-website/raw/main/content/media/SAM_26May2023/LangSAM.zip
!unzip LangSAM.zip

In [ ]:
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install -q roboflow

In [ ]:
!python pre_langsam.py



box_threshold: This value is used for object detection in the image. A higher value makes the model more selective, identifying only the most confident object instances, leading to fewer overall detections. A lower value, conversely, makes the model more tolerant, leading to increased detections, including potentially less confident ones.

text_threshold: This value is used to associate the detected objects with the provided text prompt. A higher value requires a stronger association between the object and the text prompt, leading to more precise but potentially fewer associations. A lower value allows for looser associations, which could increase the number of associations but also introduce less precise matches.

The optimal threshold can vary depending on the quality and nature of your images, as well as the specificity of your text prompts.


In [ ]:
from PIL import Image
from LangSAM import LangSAM
from demo_functions import load_ground_truth_masks , display_images_with_masks

index = '2_34'
#replace 2_36 and 2_37 for more samples
image = f'/content/{index}_img.png'
ground_truth_mask = f'/content/{index}_gt.png'

model = LangSAM()

image_pil = Image.open(image).convert("RGB")
text_prompt = "house"
masks, boxes, phrases, logits = model.predict(image_pil, text_prompt)
ground_truth_masks = load_ground_truth_masks(ground_truth_mask)

#This function displays the original image , predicted masks , and accuracy compared to ground truth
display_images_with_masks(image_pil, masks , boxes, ground_truth_mask)

Evaluation

In [ ]:
from demo_functions import MaskMatchingAlgorithm

matcher = MaskMatchingAlgorithm(ground_truth_mask, masks)
output = matcher.matching()
iou_list = output[0]
tp_pred_indices = output[1]
tp_gt_indices = output[2]
fp_indices = output[3]
fn_indices = output[4]
tp_iou_list, avg_tp_iou = matcher.tp_iou(tp_pred_indices, tp_gt_indices)
average_iou, avg_tp_iou , precision , recall , f1_score, tp_f1 = matcher.display_results(iou_list ,tp_pred_indices,
                                                                                                tp_gt_indices,
                                                                                                fp_indices,
                                                                                                fn_indices ,
                                                                                                tp_iou_list,
                                                                                                avg_tp_iou)
print('average IoU:' , average_iou)
print("average TP IoU" , avg_tp_iou)
print("precision" , precision)
print("recall" , recall)
print("F1 score" , f1_score)
print("TP F1 score" , tp_f1)